In [ ]:
# Always check your python version, most AI/ML workloads are designed around a specific version
!python --version

In [ ]:
# AWS SDK for Python, -q flag is for quiet
# This is preinstalled on SageMaker but local development you need to install boto3
%pip install -q boto3

In [ ]:
# always check your boto3 version
# Sometimes boto3 will throw errors if its out of date so check the version.
%pip show boto3

In [ ]:
# There are four different Bedrock APIs
# bedrock-runtime is for invoking GenAI models.
import boto3
client = boto3.client("bedrock-runtime")

In [ ]:
# You can reuse prompts 1-to-1 with different LLMs but for best results you need to optimize them

# This is a prompt optimized for Titan.
# Make note of User, Bot, and the {{brackets}}
# https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Text+Prompt+Engineering+Guidelines.pdf
prompt_titan = """
User: 
Sulfuric acid reacts with sodium chloride, and gives {{chemical1}} and {{chemical2}}:
Bot:
"""

# This is a prompt optimized for Claude 
# Make note of Human, Assistant and the <xml>tags</xml>
# https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/overview
# https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags
prompt_claude = """
Human: 
Sulfuric acid reacts with sodium chloride, and gives <chemical1>_____</chemical1> and <chemical2>_____</chemical2>:
Assistant:
"""

In [ ]:
import json
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/invoke_model.html

# InvokeModel directly calls the underlying API.
# InvokeModel does not standarized inputs anda outputs

# Amazon Titan Text Express is a small model
# We do not expect it to perform this task well.
# It could use fine-tuning to better form this task.
resp = client.invoke_model(
  modelId='amazon.titan-text-express-v1',
  body=json.dumps({
    "inputText": prompt_titan
  })
)
body = json.loads(resp.get("body").read())
print(body['results'][0]['outputText'])

In [ ]:
# Note how Claude 2 inputs and outputs are different from Amazon Titan Text
# Notice how Claude 2 use text completion instead of Messages API

#resp = client.invoke_model(
#    modelId='anthropic.claude-v2',
#    contentType='application/json',
#    accept='application/json',
#    body=json.dumps({
#        "prompt": prompt,
#        "max_tokens_to_sample": 200,
#        "temperature": 1.0
#    })
#)
#body = json.loads(resp.get("body").read())
#print(body['completion'])


# Notice how Claude 3 uses the Messages API
# Newer Chat Completion models *standardize on this messages API format
# https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html
# https://docs.anthropic.com/en/api/messages
resp = client.invoke_model(
    modelId='anthropic.claude-3-haiku-20240307-v1:0',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "messages": [{"role": "user", "content": [{"type": "text", "text": prompt_claude}]}],
        "max_tokens": 200,
        "temperature": 1.0,
        "anthropic_version": "bedrock-2023-05-31"
    })
)
body = json.loads(resp.get("body").read())
print(body['content'][0]['text'])

In [ ]:
# Lets create a helper.py and a text file loader
# Manging your prompt templates externally is best practice
# We wrote the helper function using Amazon Q Developer its very simple text file loader
from helper import load_text_file
prompt = load_text_file("prompts/zero-shot.txt")
print(prompt)

In [ ]:
# This is same code as before, we are just testing the external prompt
resp = client.invoke_model(
    modelId='anthropic.claude-3-haiku-20240307-v1:0',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
        "max_tokens": 200,
        "temperature": 1.0,
        "anthropic_version": "bedrock-2023-05-31"
    })
)
body = json.loads(resp.get("body").read())
print(body['content'][0]['text'])